<a href="https://colab.research.google.com/github/kakhan87/C-Sharp-Apps/blob/master/New_Doner_ANN_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle

# **Part 1 - getPredictions function**


In [ ]:
def getPredictions():
  '''This function takes in the user ID given by user,
  returns a prediction of the user ID opening the email in %tage  .
  '''
  # Load data file
  filename ='/content/drive/Shared drives/RA2 TechCareers Model Data/pkl_folder/base_features_ANN.pkl'
  data = pickle.load(open(filename, 'rb'))


  # Loading NN model from disk

  from numpy import loadtxt
  from keras.models import load_model
  
  loaded_model = load_model('/content/drive/Shared drives/RA2 TechCareers Model Data/pkl_folder/New_Donor_ANN_Model.h5')

  #Importing LDA data
  %cd /content/drive/Shared drives/RA2 TechCareers Model Data/New_Donor_Prediction
  %run 'newEmail_Topic_Modeling.ipynb' import newContentScore

  #Saving column sequence of trained data
  seq = (data.columns.values).tolist()


  # Sorting dataset w.r.t. User IDs and email sending time
  data_sorted = data.sort_values(['user_id', 'sent_time'], ascending=False).reset_index(drop=True)

  # Drop dublicate User IDs and keep one with most recent email sent
  data_dropped = data_sorted.drop_duplicates(subset='user_id', keep='first', inplace=False)


  #Shift data
  data_shifted = data_dropped.drop(['Opened_L5'], axis=1)
  data_shifted = data_shifted.rename(columns={'Opened_L4': 'Opened_L5', 
                                              'Opened_L3': 'Opened_L4', 
                                              'Opened_L2': 'Opened_L3', 
                                              'Opened_L1': 'Opened_L2', 
                                              'opened': 'Opened_L1'})

  data_shifted = data_dropped.drop(['T4_L5', 'T3_L5', 'T2_L5', 'T1_L5', 'T0_L5'], axis=1)

  for i in range (4,-1,-1):
    for j in range (5,0,-1):
      new_name = ('T'+str(i)+'_L'+str(j-1))
      old_name = ('T'+str(i)+'_L'+str(j))
      data_shifted = data_shifted.rename(columns={new_name:old_name})
    new_name = ('content_topic_'+str(i))
    data_shifted = data_shifted.rename(columns={new_name:old_name})


  #Initializing prediction list for user IDs
  user_pred = []
  #Getting list of user IDs to make predictions
  user_list = data['user_id'].to_numpy().tolist()
  # user_list = [5545, 13658, 18215]
  #Looping over each user to make preidctions
  for user in user_list:

    # Creating dataframe for one single user at a time
    # data_input = data_shifted.loc[data_shifted['user_id'].isin(user)]
    data_input = data_shifted[data_shifted['user_id'] == user]

    #Adding user ID to new content score
    newScore = newContentScore.assign(user_id = [user]) 

    #Merging Content topic 
    data_joined = pd.merge(data_input, newScore, on='user_id', how='inner')

    #Dropping user ID from new content score
    newScore = newScore.drop(['user_id'], 1)

    #Arranging column
    data_ordered = data_joined.reindex(columns=seq)



    #Dropping unnecessary columns from new user data
    X_test = data_ordered.drop(['email_id', 'user_id', 'sent_time', 'opened'], 1)


    # Making the predictions
    y_pred = np.array(loaded_model.predict(X_test))[0][0].tolist()
    user_pred.append(y_pred)  

  print(user_pred)
  def user_pred_histogram (y_pred_list, user_list):

    data_plot = pd.DataFrame({"Prediction":y_pred_list, "User_List":user_list})
    data_plot.set_index('User_List', inplace=True)

    # np.histogram returns 2 values
    count, bin_edges = np.histogram(data_plot)
    bin_edges = np.round(bin_edges,2) # Round bin_edges to 1 decimal place 
    # print("Frequency Count:", count) # frequency count
    # print("Bins:", bin_edges) # bin ranges, default = 10 bins

    # Density Plot and Histogram of all arrival delays
    plt.figure(figsize=(10, 6))
    sns.distplot(data_plot, hist=True, kde=False, 
                bins=bin_edges, color = 'darkblue', 
                hist_kws={'edgecolor':'black'})

    plt.title('Histogram of Predicted Number of Users Opening a New E-mail') # add a title to the histogram
    plt.ylabel('Number of Users') # add y-label
    plt.xlabel('Likelihood of Opening New E-mail') # add x-label

    plt.rcParams.update({'font.size': 18})
    plt.show()

  user_pred_histogram(user_pred, user_list)

  # return user_pred


In [ ]:
getPredictions()